In [32]:
import nibabel as nib
import numpy as np
import pandas as pd
from nilearn.image import resample_to_img
from nilearn import plotting
import os


In [33]:
# ============
# paths
# ============
repo_path="/Users/parri/OneDrive/Documentos/Beca PEFI/brainage-models-benchmark"

atlas_path = repo_path+"/utils/Hammers_mith-n30r95-MaxProbMap-full-MNI152-SPM12.nii.gz" #Atlas path
labels_path = repo_path+"/utils/labels_Hammers.csv" #Atlas labels path
lrp_path=repo_path+'/results/LRP_saliency_maps/BrainAgeNeXt/lrp_mean_UNSAM_CN.nii.gz'

In [34]:
# -------------------
# Load Images
# -------------------
lrp_img=nib.load(lrp_path)
atlas_img = nib.load(atlas_path)
labels_df = pd.read_csv(labels_path)

label_values = labels_df["n_label"].tolist()
label_names  = labels_df["structure"].tolist()

# Fix atlas shape (if 4D)
atlas_data = atlas_img.get_fdata()
if atlas_data.ndim == 4 and atlas_data.shape[-1] == 1:
    atlas_img = nib.Nifti1Image(np.squeeze(atlas_data, -1), atlas_img.affine, atlas_img.header)

# Resample to LRP space
atlas_resampled = resample_to_img(atlas_img, lrp_img, interpolation="nearest")
atlas_data = atlas_resampled.get_fdata()

# Ensure 3D
if atlas_data.ndim == 4 and atlas_data.shape[-1] == 1:
    atlas_data = np.squeeze(atlas_data, -1)

lrp_data = lrp_img.get_fdata()

# -------------------
# compute explainability scores per region
# -------------------
region_scores = {}

for roi_val, roi_name in zip(label_values, label_names):
    mask = atlas_data == roi_val
    vals = lrp_data[mask]
    region_scores[roi_name] = np.mean(vals) if vals.size > 0 else np.nan

df_scores = pd.DataFrame.from_dict(region_scores, orient="index", columns=["explainability"])


C:\Users\parri\AppData\Local\Temp\ipykernel_24200\1742387050.py:17: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  atlas_resampled = resample_to_img(atlas_img, lrp_img, interpolation="nearest")
C:\Users\parri\AppData\Local\Temp\ipykernel_24200\1742387050.py:17: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  atlas_resampled = resample_to_img(atlas_img, lrp_img, interpolation="nearest")
C:\Users\parri\AppData\Local\Temp\ipykernel_24200\1742387050.py:17: UserWarning: The provided image has no sform in its header. Please check the provided file. Results may not be as expected.
  atlas_resampled = resample_to_img(atlas_img, lrp_img, interpolation="nearest")


In [ ]:
#df_scores.to_csv("lrp_region_scores_UNSAM.csv")  # save results in a csv

In [11]:
#read al csv files previously saved
adniscores=pd.read_csv("../data/Relevance_scoreboard/BrainAgeNeXt/affine/lrp_region_scores_ADNI_CN.csv")
ncnscores=pd.read_csv("../data/Relevance_scoreboard/BrainAgeNeXt/affine/lrp_region_scores_JUK.csv")
cogconvarscores=pd.read_csv("../data/Relevance_scoreboard/BrainAgeNeXt/affine/lrp_region_scores_RRIB.csv")
cpscores=pd.read_csv("../data/Relevance_scoreboard/BrainAgeNeXt/affine/lrp_region_scores_UNSAM_CONTROLS.csv")

In [14]:
#normalize intensities by its maximum voxel's relevance score
adniscores['Explicabilidad']=adniscores['Explicabilidad']/10552
ncnscores['Explicabilidad']=ncnscores['Explicabilidad']/9486
cogconvarscores['Explicabilidad']=cogconvarscores['Explicabilidad']/7723
cpscores['Explicabilidad']=cpscores['Explicabilidad']/7670


In [26]:
adniscores.rename(columns={"Unnamed: 0":"Region"}, inplace=True)
ncnscores.rename(columns={"Unnamed: 0":"Region"}, inplace=True)
cogconvarscores.rename(columns={"Unnamed: 0":"Region"}, inplace=True)
cpscores.rename(columns={"Unnamed: 0":"Region"}, inplace=True)

In [27]:
ncnscores

,Region,Explicabilidad
0,Lateral-ventricle-excluding-temporal-horn-R,0.425098
1,pallidum-R,0.286027
2,insula-posterior-long-gyrus-R,0.284125
3,thalamus-R,0.274340
4,insula-anterior-long-gyrus-R,0.266771
...,...,...
90,OL-lateral-remainder-occipital-lobe-R,-0.064123
91,PL-supramarginal-gyrus-L,-0.066691
92,OL-cuneus-L,-0.078323
93,PL-angular-gyrus-L,-0.092348


In [28]:
# summatory of explainability scores
final_scores = (
    adniscores.set_index("Region")["Explicabilidad"] +
    cogconvarscores.set_index("Region")["Explicabilidad"] +
    cpscores.set_index("Region")["Explicabilidad"] +
    ncnscores.set_index("Region")["Explicabilidad"]
).reset_index()

final_scores.columns = ["Region", "Total Explicabilidad"]

final_scores = final_scores.sort_values(by="Total Explicabilidad", ascending=False)
final_scores

,Region,Total Explicabilidad
29,Lateral-ventricle-excluding-temporal-horn-R,1.605041
28,Lateral-ventricle-excluding-temporal-horn-L,1.178074
66,Third-ventricle,1.093837
72,corpus-callosum,1.079632
94,thalamus-R,0.993903
...,...,...
44,PL-supramarginal-gyrus-L,-0.423940
8,FL-lateral-orbital-gyrus-L,-0.430322
48,TL-anterior-temporal-lobe-lateral-part-L,-0.465420
39,PL-angular-gyrus-R,-0.478769


In [ ]:
#final_scores.to_csv("lrp_region_scores_total.csv", index=False)